# Connecting the Retina and the Lamina LPUs through Neurokernel Interface

We demonstrate in this notebook how to plug in individual LPUs into a Neurokernel simulation, and connect these LPUs through the Neurokernel interface. The two LPUs that we use here are the models for the retina and the lamina, the first two LPUs in the visual sytem of the fruit fly. According to the Neurokernel API, a successful integration of these LPUs requires each of them to expose named ports. We briefly summarize the ports exposed by the two LPUs first, and then detail the connection between them. This notebook is meant to be narrative, but not directly executable due to MPI limitation. For executable example, see [examples/retlam_demo/retlam_demo.py](https://github.com/neurokernel/retina-lamina/blob/master/examples/retlam_demo/retlam_demo.py) in the [neurokernel/retina-lamina repository](https://github.com/neurokernel/retina-lamina) for integrating the retina and lamina models.

## Configuration

Before simulation we need to provide a configuration of various parameters of both LPUs. The configuration is assumed to have the form of a dictionary. The configuration combines those in the retina and the lamina modules. For easier manipulation this configuration can be read from a configuration file. Details of each parameter can be found in the configuration template in [examples/template_spec.cfg](https://github.com/neurokernel/retina-lamina/blob/master/examples/template_spec.cfg).

In [ ]:
config = {}
config['General'] = {}
config['General']['dt'] = 1e-4
config['General']['steps'] = 1000
    
config['Retina'] = {}
config['Retina']['model'] = 'vision_model_template'
config['Retina']['acceptance_factor'] = 1
config['Retina']['screentype'] = 'Sphere'
config['Retina']['filtermethod'] = 'gpu'
config['Retina']['intype'] = 'Bar'
config['Retina']['time_rep'] = 1
config['Retina']['space_rep'] = 1

config['Lamina'] = {}
config['Lamina']['model'] = 'vision_model_template'
config['Lamina']['relative_am'] = 'half'

config['Screen'] = {}
config['Screen']['SphereScreen'] = {}
config['Screen']['SphereScreen']['parallels'] = 50
config['Screen']['SphereScreen']['meridians'] = 100
config['Screen']['SphereScreen']['radius'] = 10
config['Screen']['SphereScreen']['half'] = False
config['Screen']['SphereScreen']['image_map'] = 'AlbersProjectionMap'

config['InputType'] = {}
config['InputType']['shape'] = [100, 100]
config['InputType']['infilename'] = ''
config['InputType']['writefile'] = False

config['InputType']['Bar'] = {}
config['InputType']['Bar']['bar_width'] = 10
config['InputType']['Bar']['direction'] = 'v'
config['InputType']['Bar']['levels'] = [3e3, 3e4]
config['InputType']['Bar']['speed'] = 1000
config['InputType']['Bar']['double'] = False

## Creation of Neurokernel Manager

We first create the Neurokernel manager.

In [ ]:
import neurokernel.core_gpu as core
import neurokernel.mpi_relaunch

manager = core.Manager()

## The Retina and its Interface

An input stimulus generator is needed to generate the inputs during simulation.

In [ ]:
from retina.input_generator import RetinaInputGenerator

generator = RetinaInputGenerator(config)

We construct the retina LPU from the retina module and add it to the Neurokernel simulation. We follow closely the retina example in which the retina LPU is executed in isolation. 

In [ ]:
import networkx as nx
import retina.geometry.hexagon as ret_hx
import retina.retina as ret
from retina.LPU import LPU as rLPU

# create a hexagonal array for the retina
# This describes the array of positions of neurons, their arrangement in space and a way to query for neighbors 
ret_hexagon = ret_hx.HexagonArray(num_rings=14, radius=1)
# create a retina object that contains neuron and synapse information
retina_array = ret.RetinaArray(ret_hexagon, config)
# add the retina object to the stimulus generator
generator.retina = retina_array

# parameters from the configuration dictionary
dt = config['General']['dt']

output_file = 'retina_output.h5'
gexf_file = 'retina.gexf.gz'

G = retina_array.get_worker_nomaster_graph()
# export the configuration of neurons and synapses to a GEXF file
nx.write_gexf(G, gexf_file)
# parse GEXF file
n_dict_ret, s_dict_ret = rLPU.lpu_parser(gexf_file)
retina_id = 'retina0'
modules = []

# add the retina LPU to Neurokernel manager
manager.add(rLPU, retina_id, dt, n_dict_ret, s_dict_ret,
            input_file=None, output_file=output_file,
            device=0, debug=False, time_sync=False,
            modules=modules, input_generator=generator)

The Retina model ([Neurokernel RFC #3](http://neurokernel.github.io/docs.html)) exposes their outputs, the photoreceptors R1-R6, to the Neurokernel interface. The naming convention of the ports is as follows: A port associated with a photoreceptor is named as `/ret/<omm_id>/<photor_name>`, where `omm_id` is a unique numeric identifier of the ommatidium that the photoreceptor resides, and `photor_name` the name of photoreceptor.

## The Lamina and its Interface

We construct the lamina LPU from the lamina module and add it to the Neurokernel simulation. We follow closely the lamina example in which the lamina LPU is executed in isolation.

In [ ]:
import lamina.geometry.hexagon as lam_hx
import lamina.lamina as lam
from lamina.LPU import LPU as lLPU

# create a hexagonal array for the lamina
lam_hexagon = lam_hx.HexagonArray(num_rings=14, radius=1)
# create a lamina object that contains neuron and synapse information
lamina_array = lam.LaminaArray(lam_hexagon, config)

# parameters from the configuration dictionary
dt = config['General']['dt']

output_file = 'lamina_output.h5'
gexf_file = 'lamina.gexf.gz'
G = lamina_array.get_graph()
# export the configuration of neurons and synapses to a GEXF file
nx.write_gexf(G, gexf_file)
# parse GEXF file
n_dict_ret, s_dict_ret = lLPU.lpu_parser(gexf_file)
lamina_id = 'lamina0'
modules = []

# add the lamina LPU to Neurokernel manager
manager.add(lLPU, lamina_id, dt, n_dict_ret, s_dict_ret,
            input_file=None, output_file=output_file,
            device=1, debug=False, time_sync=False,
            modules=modules, input_generator=None)

Photoreceptors R1-R6 consititue the inputs to the lamina LPU from the retina LPU. The input ports in the lamina LPU follows the naming convention: a photoreceptor input port is named as `/lam/<cart_id>/<photor_name>` where `cart_id` is the unique numeric identifier of the cartridge that the input port belongs to, and `photor_name` is the name of the photoreceptor. Note that retinotopy in the early visual system of the fruit fly is imposed by the hexagonal array of ommatidia in the retina and that of cartridges in the lamina. The two arrays are assumed to be compatible to each other, i.e., the ommatidia have a one-to-one correspondence to the cartridges.

## Connection between the Retina and the Lamina

The connections between retina and lamina follow the neural superposition rule of the fly's compound eye (see [Neurokernel RFC#2](http://neurokernel.github.io/docs.html)). The superposition rule is also illustrated in the figure below, where the solid circles represent ommatidia in the retina, and dashed circles represent cartridges in the lamina. Individual photoreceptors R1-R6 are numbered and their relative positon highlighted in some of the ommatidia. On the left, cartridge A receives 6 photoreceptor inputs, each from a different ommatidium. On the right, 6 photoreceptors from a single ommatidium each projects to a different cartridge.
<img src='files/files/neural_superposition.png'>

The two LPUs added to the Neurokernel have properly exposed their I/O to the Neurokernel interface. The connection between the two LPUs can then be configured through the Pattern provided by the Neurokernel API.

In [ ]:
from neurokernel.pattern import Pattern

retina_selectors = retina_array.get_all_selectors()
lamina_selectors = lamina_array.get_all_selectors()

pattern = Pattern(','.join(retina_selectors), ','.join(lamina_selectors))

rulemap = retina_array.rulemap
for ret_sel in retina_selectors:
    # format should be '/ret/<omm_id>/<photor_name>'
    _, lpu, ommid, n_name = ret_sel.split('/')

    # find neighbor of neural superposition
    neighborid = rulemap.neighbor_for_photor(int(ommid), n_name)

    # format should be '/lam/<cart_id>/<photor_name>'
    lam_sel = lamina_array.get_selector(neighborid, n_name)

    pattern.interface[ret_sel, 'type'] = 'gpot'
    pattern.interface[lam_sel, 'type'] = 'gpot'
    
    # src, dest
    pattern[ret_sel, lam_sel] = 1

Finally, we connect the retina and the lamina LPUs using the pattern.

In [ ]:
manager.connect(retina_id, lamina_id, pattern, compat_check=False)

The following figure schematically shows the integrated simulation of the retina and the lamina LPUs. The retina and the lamina both expose their ports to the Neurokernel through the LPU interface, and Neurokernel handles the communication between the two LPUs.
<img src='files/files/retlam.png'>

Simulation of the connected retina and lamina LPUs can then be started

In [ ]:
steps = config['General']['steps']
manager.spawn()
manager.start(steps=steps)
manager.wait()